<a href="https://colab.research.google.com/github/dlsys10714/notebooks/blob/main/17_hardware_acceleration_architecture_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 17: Hardware Acceleration Architecture Overview 

In this lecture, we will to walk through backend scafoldings to get us hardware accelerations for needle.




## Select a GPU runtime type
In this lecture, we are going to make use of c++ and CUDA to build accelerated linear algebra libraries. In order to do so, please make sure you select a runtime type with GPU and rerun the cells if needed:
- Click on the "Runtime" tab
- Click "Change runtime type"
- Select GPU

After you started the right runtime, you can run the following command to check if there is a GPU available.

In [1]:
!nvidia-smi

Mon Nov  1 14:37:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Prepare the codebase

To get started, we can clone the related lecture13 repo from the github. 

In [2]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p 10714
%cd /content/drive/MyDrive/10714
# comment out the following line if you run it for the second time
# as you already have a local copy of lecture17
# !git clone https://github.com/dlsys10714/lecture17
!ln -s /content/drive/MyDrive/10714/lecture17 /content/needle

Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/10714


In [3]:
!python3 -m pip install pybind11

     |████████████████████████████████| 208 kB 5.3 MB/s 


### Build the needle cuda library

We leverage pybind to build a c++/cuda library for acceleration. You can type make to build the corresponding library.

In [ ]:
%cd /content/needle
!make

/content/drive/MyDrive/10714/lecture17
-- Found pybind11: /usr/local/lib/python3.7/dist-packages/pybind11/include (found version "2.8.1" )
-- Found cuda, building cuda backend
-- Autodetected CUDA architecture(s):  3.7
-- Configuring done
-- Generating done
-- Build files have been written to: /content/drive/MyDrive/10714/lecture17/build
make[1]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[2]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[3]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
[-25%] Building NVCC (Device) object CMakeFiles/ndarray_backend_cuda.dir/src/ndarray_backend_cuda_generated_ndarray_backend_cuda.cu.o
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_5

We can then run the following command to make the path to the package available in colab's environment as well as the PYTHONPATH.

In [4]:
%set_env PYTHONPATH /content/needle/python:/env/python
import sys
sys.path.append("/content/needle/python")

env: PYTHONPATH=/content/needle/python:/env/python


## Codebase walkthrough


Now click the files panel on the left side. You should be able to see these files

Python:
- needle/backend_ndarray/ndarray.py
- needle/backend_ndarray/ndarray_backend_numpy.py

C++/CUDA
- src/ndarray_backend_cpu.cc
- src/ndarray_backend_cuda.cu

The main goal of this lecture is to create an accelerated ndarray library.
As a result, we do not need to deal with needle.Tensor for now and will focus on backend_ndarray's implementation. 

After we build up this array library, we can then use it to power backend array computations in needle.


## Creating a CUDA NDArray






In [5]:
from needle import backend_ndarray as nd

In [6]:
x = nd.NDArray([1, 2, 3])

In [7]:
y = x + x

In [8]:
y

NDArray([2. 4. 6.], device=numpy_device())

We can create a CUDA tensor from the data by specifying a device keyword.

In [9]:
x = nd.NDArray([1, 2, 3], device=nd.cuda())

In [10]:
y = x + 1

In [11]:
x.numpy()

array([1., 2., 3.], dtype=float32)

In [12]:
x.device

cuda()

In [13]:
y = x + 1

In [14]:
y.device

cuda()

In [15]:
y.numpy()

array([2., 3., 4.], dtype=float32)

### Key Data Structures

Key data structures in backend_ndarray

- NDArray: the container to hold device specific ndarray
- BackendDevice: backend device
    - mod holds the module implementation that implements all functions
    - checkout ndarray_backend_numpy.py for a python-side reference.



## Trace GPU execution

Now, let us take a look at what happens when we execute the following code


In [16]:
x = nd.NDArray([1, 2, 3])
y = x + 1

Have the following trace:

backend_ndarray/ndarray.py
- `NDArray.__add__`
- `NDArray.ewise_or_scalar`
- `ndarray_backend_cpu.cc:ScalarAdd`

In [17]:
y.numpy()

array([2., 3., 4.], dtype=float32)

Have the following trace:

- `NDArray.numpy`
- `ndarray_backend_cpu.cc:to_numpy`

## Guidelines for Reading C++/CUDA related Files

Read
- src/ndarray_backend_cpu.cc
- src/ndarray_backend_cuda.cu


Optional
- CMakeLists.txt: this is used to setup the build and likely you do not need to tweak it.







## NDArray Data Structure

Open up `python/needle/backend_ndarray/ndarray.py`.

An NDArray contains the following fields:
- handle: The backend handle that build a flat array which stores the data.
- shape: The shape of the NDArray
- strides: The strides that shows how do we access multi-dimensional elements
- offset: The offset of the first element.
- device: The backend device that backs the computation






## Transformation as Strided Computation

We can leverage the strides and offset to perform transform/slicing with zero copy.

- Broadcast: insert strides that equals 0
- Tranpose: swap the strides
- Slice: change the offset and shape 

For most of the computations, however, we will call `array.compact()` first to get a contiguous and aligned memory before running the computation.

In [18]:
x = nd.NDArray([1, 2, 3, 4], device=nd.numpy_device())

In [19]:
x.numpy()

array([1., 2., 3., 4.], dtype=float32)

We can use strides and shape manipulation to create different views of the same array.

In [20]:
y = nd.NDArray.make(shape=(2, 2), strides=(2, 1), device=x.device, handle=x._handle, offset=0)

In [21]:
y.numpy()

array([[1., 2.],
       [3., 4.]], dtype=float32)

In [22]:
z = nd.NDArray.make(shape=(2, 1), strides=(2, 1), device=x.device, handle=x._handle, offset=1)

In [23]:
z.numpy()

array([[2.],
       [4.]], dtype=float32)

## CUDA Acceleration

Now let us open `src/ndarray_cuda_backend.cu` and take a look at current implementation of GPU ops.


## Steps for adding a new operator implementation
- Add an implementation in `ndarray_backend_cuda.cu`, expose via pybind
- Call into the operator in ndarray.py
- Write up testcases

In [ ]:
!make

-- Found pybind11: /usr/local/lib/python3.7/dist-packages/pybind11/include (found version "2.8.1" )
-- Found cuda, building cuda backend
-- Autodetected CUDA architecture(s):  3.7
-- Configuring done
-- Generating done
-- Build files have been written to: /content/drive/MyDrive/10714/lecture17/build
make[1]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[2]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[3]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[3]: Leaving directory '/content/drive/MyDrive/10714/lecture17/build'
make[3]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
[-25%] Linking CXX shared module ../python/needle/backend_ndarray/ndarray_backend_cuda.cpython-37m-x86_64-linux-gnu.so
make[3]: Leaving directory '/content/drive/MyDrive/10714/lecture17/build'
[  0%] Built target ndarray_backend_cuda
make[3]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[3]: Leaving

## Connect back to needle Tensor

So far we only played with the `backend_ndarray` subpackage, which is a self-contained ndarray implementation within needle.

We can connect the ndarray back to needle as a backend.

In [24]:
import needle as ndl

In [25]:
x = ndl.Tensor([1,2,3], device=ndl.cuda(), dtype="float32")
y = ndl.Tensor([2,3,5], device=ndl.cuda(), dtype="float32")
z = x + y

In [26]:
z.device

cuda(0)

To see how the operation get registered. Open and read the following file

- python/needle/nd_backend.py

Note that the backend computation registration `register_nd_compute` is only needed because there may be a implementation difference between numpy and backend ndarray. If all the ndarray follows exactly same set of API, we can also directly remove this layer of indirection.

Try to put things together and try out the autograd implementation, this time with the cuda backend.

In [27]:
z.backward()

In [28]:
x.grad

needle.Tensor([1. 1. 1.])

In [ ]:
!nvprof python tests/test_backend_ndarray.py

## Write Standalone Python Test Files

Now that we have additional c++/cuda libraries in needle, we will need to type make in order to rebuild the library. Additionally, because the colab environment caches the old library, it is inconvenient to use the ipython cells to debug the updated library.




In [ ]:
!make

-- Found pybind11: /usr/local/lib/python3.7/dist-packages/pybind11/include (found version "2.8.1" )
-- Found cuda, building cuda backend
-- Autodetected CUDA architecture(s):  3.7
-- Configuring done
-- Generating done
-- Build files have been written to: /content/drive/MyDrive/10714/lecture17/build
make[1]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[2]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[3]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[3]: Leaving directory '/content/drive/MyDrive/10714/lecture17/build'
[  0%] Built target ndarray_backend_cuda
make[3]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[3]: Leaving directory '/content/drive/MyDrive/10714/lecture17/build'
[ 50%] Built target ndarray_backend_cpu
make[2]: Leaving directory '/content/drive/MyDrive/10714/lecture17/build'
make[1]: Leaving directory '/content/drive/MyDrive/10714/lecture17/build'



We recommend writing separate python files and invoke them from the command line. Create a new file `tests/mytest.py` and write your local tests. This is also a common develop practice in big projects that involves python c++ FFI.

In [ ]:
!python tests/mytest.py

python3: can't open file 'tests/mytest.py': [Errno 2] No such file or directory


After we have building the library, we could choose to fully restart the runtime (factory reset runtime) if you want to bring the updated change back to another colab. Note that you will need to save your code changes to the drive or a private github repo.